## 4 树

### 4.1 B树

> 前置知识
>
> 度数：树中某一节点的孩子数目。
>
> 阶数：树的所有节点孩子数目的最大值。

B 树即 balanced tree，平衡树。B树的每个节点可以有多个子节点，且每个节点都包含一个有序的键值列表。B树的特点是：
- 每个节点最多有 r 个孩子（r 称为 B 树的阶）。
- 所有叶子节点都在同一层上。
- 除了根节点和叶子节点外，每个节点至少有 $ceil(\frac{r}{2})$ 个孩子。
- 根节点至少有两个孩子（除非树为空）。
- 每个非叶子节点由 $[ceil(\frac{r}{2})-1, r-1)$ 个键值和 $[ceil(\frac{r}{2}), r]$ 个孩子组成。
    - 也就是说，每个节点的键值数目不能大于或等于树的阶。
    - 孩子数目和阶的数目紧密相关。
- 难以言表，看图

<img src="./btree.png" alt="b-tree" style="zoom:50%;" />



In [5]:
class BTreeNode:
    keys: list[int]
    children: list["BTreeNode"]
    is_leaf: bool
    key_count: int
    min_children: int
    max_children: int  # max_children = r，r 为 B 树的阶数

    def __init__(self, max_children: int, keys: list[int] = None, children: list["BTreeNode"] = None,
                 is_leaf: bool = True):
        self.max_children = max_children
        self.is_leaf = is_leaf
        self.keys = keys or []
        self.children = children or []
        self.key_count = len(self.keys)

    def get(self, key: int):
        """多路查找指定的 key"""
        # 1 找到第一个 key 大于等于 key 的索引
        index = 0
        while index < self.key_count:
            if key == self.keys[index]:
                return self
            elif key > self.keys[index]:
                break
            index += 1
        # 2.1 遍历完成，若当前节点为叶子节点，无法找到，返回 None
        if self.is_leaf:
            return None
        # 2.2 遍历完成，若当前节点为非叶子节点，递归查找对应的子节点
        return self.children[index].get(key)

    def insert_key(self, key: int, index: int):
        """插入 key"""
        self.keys.insert(index, key)
        self.key_count += 1

    def insert_child(self, child: "BTreeNode", index: int):
        """插入子节点"""
        self.children.insert(index, child)

    def remove_key(self, key: int):
        self.keys.pop(self.keys.index(key))




In [10]:
from typing import Literal


class BTree:
    root: BTreeNode
    min_key_count: int  # min_key_count = ceil(r/2) - 1
    max_key_count: int  # max_key_count = r - 1
    min_children: int  # max_children = r // 2
    max_children: int  # max_children = r，r 为 B 树的阶数

    def __init__(self, max_children: int = 4, root: BTreeNode = None):
        self.root = root
        self.max_children = max_children
        self.min_children = max_children // 2
        self.min_key_count = max_children // 2 - 1
        self.max_key_count = max_children - 2

    def __contains__(self, item: int):
        return self.root.get(item) is not None

    def put(self, key: int):
        def _recursive_put(node_: BTreeNode, parent_: BTreeNode | None, index_in_parent_: int):
            index = 0
            for i, k in enumerate(node_.keys):
                if k == key:
                    return  # 此处简化，实际上应该要在这里更新值
                if k > key:
                    index = i
                    break
            if node_.is_leaf:
                # 如果当前节点实际上为叶子节点，直接插入到 keys 中
                node_.keys.index(index, key)
            else:
                # 如果找到的是对应的孩子，让孩子参与递归，插入到孩子中
                _recursive_put(node_.children[index], node_, index)
            # 插入完成后判断要不要分裂
            if node_.key_count >= self.max_key_count:
                self.split(node_, parent_, index_in_parent_)

        _recursive_put(self.root, None, 0)

    def remove(self, key: int, start: BTreeNode = None, parent: BTreeNode = None, index_in_parent: int = 0):
        def _recursive_remove(node_: BTreeNode, parent_: BTreeNode | None, index_in_parent_: int):
            # 1 遍历当前 node 的 keys 列表
            index = 0
            for i, k in enumerate(node_.keys):
                if k >= key:
                    index = i
                    break
                i += 1
            if node_.is_leaf:
                if index < node_.key_count and key in node_.keys:
                    # 情况1：叶子，在 keys 中找到，直接删除
                    node_.keys.pop(index)
                else:
                    # 情况2：叶子，在 keys 中没找到，直接返回
                    return
            else:
                if index < node_.key_count and key in node_.keys:
                    # 情况3：非叶子，在 keys 中找到，要把当前的 keys 替换掉（十分十分类似于二叉搜索树结点有孩子的删除逻辑）
                    # 1 找到第 index + 1 子树的最小值
                    temp_node = node_.children[index + 1]
                    while not temp_node.is_leaf:
                        temp_node = temp_node.children[0]
                    # 取最小的key
                    replacer = temp_node.keys[0]
                    # 2 把 replacer 从第 index + 1 子树中删掉
                    self.remove(replacer, node_.children[index + 1], node_, index + 1)
                    # 3 换上去
                    node_.keys[index] = replacer
                else:
                    # 情况4：非叶子，在 keys 中没找到，向第 index 个 children 中查找
                    _recursive_remove(node_.children[index], node_, index)
            # 情况5/6：重新调整平衡
            if node_.key_count < self.min_key_count:
                self.balance(node_, parent_, index_in_parent_)

        start = start or self.root
        _recursive_remove(start, parent, index_in_parent)

    def get_siblings(self, node: BTreeNode | int, parent: BTreeNode, of: Literal["left", "right"]):
        if isinstance(node, BTreeNode):
            for i, c in enumerate(parent.children):
                if c is node:
                    node = i
                    break
        if of == "left":
            return parent.children[node - 1] if node >= 0 else None
        else:
            return parent.children[node + 1] if node < len(parent.children) else None

    def balance(self, node: BTreeNode, parent: BTreeNode, index_in_parent: int):
        if self.root is node:
            # 情况6：如果根节点就是不平衡的，孩子不为空的情况下让自己的孩子做根节点
            if node.key_count == 0 and node.children:
                self.root = node.children[0]
            return
        left_sib = self.get_siblings(index_in_parent, parent, "left")
        right_sib = self.get_siblings(index_in_parent, parent, "right")
        if left_sib and left_sib.key_count > self.min_key_count:
            # 情况5-1：左边富裕，右旋
            # 1 将父节点的前一个 key 作为不平衡结点的第一个 key
            node.insert_key(parent.keys[index_in_parent - 1], 0)
            # 2 将左节点最后一个 key 换到父节点的缺失位置上
            parent.keys[index_in_parent - 1] = left_sib.keys.pop(-1)
            # 3 要是左节点有孩子，让最右边的孩子当不平衡结点的第0个孩子
            if not left_sib.is_leaf:
                node.insert_child(parent.children.pop(-1), 0)
        elif right_sib and right_sib.key_count > self.min_key_count:
            # 情况5-2：右边富裕，左旋
            # 1 将父节点的后一个 key 作为不平衡结点的最后 key
            node.insert_key(parent.keys[index_in_parent], node.key_count)
            # 2 将右节点最左一个 key 换到父节点的缺失位置上
            parent.keys[index_in_parent] = right_sib.keys.pop(0)
            # 3 要是右节点有孩子，让最右边的孩子当不平衡结点的第0个孩子
            if not right_sib.is_leaf:
                node.insert_child(parent.children.pop(-1), node.key_count + 1)
        else:
            # 情况5-3：左右都不富裕，向左合并
            if left_sib:
                # 1 在父节点中移除不平衡的节点
                parent.children.pop(index_in_parent)
                # 2 将父节点中前一个 key 给左节点，左节点放到最后面
                left_sib.insert_key(parent.keys.pop(index_in_parent - 1), left_sib.key_count)
                # 3 不平衡结点全部接到左孩子中
                left_sib.keys += node.keys
                if not node.is_leaf:
                    left_sib.children += node.children
            else:
                # 情况5-4：没有左边，向自己合并
                # 1 在父节点中移除右兄弟节点
                parent.children.pop(index_in_parent + 1)
                # 2 将父节点中后一个 key 给自己节点，自己放到最后面
                node.insert_key(parent.keys.pop(index_in_parent), node.key_count)
                # 3 右结点全部接到自己中
                node.keys += right_sib.keys
                if not node.is_leaf:
                    node.children += right_sib.children

    def split(self, node: BTreeNode, parent: BTreeNode, index_in_parent: int):
        if self.root is node:
            # 特例1：需要分裂的是根节点
            new_root = BTreeNode(
                max_children=self.max_children,
                children=[node],
                is_leaf=False
            )
            self.root = new_root
            node.is_leaf = True
            parent = new_root
            index_in_parent = 0
        # 1 创建新的右节点
        new_node: BTreeNode = BTreeNode(
            self.max_children,
            keys=node.keys[self.min_children: 2 * self.min_children - 1],
            is_leaf=node.is_leaf
        )
        node.keys = node.keys[:self.min_children]
        new_node.key_count = self.min_children - 1  # 分裂时新节点取的节点数量就是这么多
        node.key_count = self.min_children - 1  # 除去推出的多余的 key 外，还要推出一个 key 给中间的结点（这个中间在下一次中推出）
        if not node.is_leaf:
            # 如果不是 node 叶子结点，还要挪 children 列表
            new_node.children = node.children[self.min_children + 1:]
            node.children = node.children[:self.min_children + 1]
        # 2 把中间值插入父节点
        parent.insert_key(node.keys.pop(), index_in_parent)
        # 3 更新父节点
        parent.insert_child(new_node, index_in_parent + 1)  # 在旧结点 +1 的位置放新节点


In [12]:
test_rank = 6
test_btree1 = BTree(
    test_rank,
    root=BTreeNode(
        test_rank,
        keys=[3],
        children=[BTreeNode(test_rank, keys=[1, 2]), BTreeNode(test_rank, keys=[4, 5, 6, 7, 8])],
        is_leaf=False
    )
)
test_btree1.split(test_btree1.root.children[1], test_btree1.root, 1)

In [13]:
test_btree2 = BTree(
    test_rank,
    root=BTreeNode(
        test_rank,
        keys=[1, 2, 3, 4, 5],
        is_leaf=True
    )
)
test_btree2.split(test_btree2.root, None, 0)

In [14]:
test_btree3 = BTree(
    test_rank,
    root=BTreeNode(
        test_rank,
        keys=[3],
        children=[
            BTreeNode(test_rank, keys=[1, 2]), BTreeNode(test_rank, keys=[5, 7, 9, 11, 13], is_leaf=False, children=[
                BTreeNode(test_rank, [4]), BTreeNode(test_rank, [6]), BTreeNode(test_rank, [8]),
                BTreeNode(test_rank, [10]), BTreeNode(test_rank, [12]), BTreeNode(test_rank, [14])
            ])
        ],
        is_leaf=False
    )
)
test_btree3.split(test_btree3.root.children[1], test_btree3.root, 1)